# Trabalho de paralelismo
### Simulação de Monte Carlo para aproximação de PI
--- 

## Introdução ao problema

A simulação de Monte Carlo é uma técnica estatística que utiliza amostragem aleatória para prever resultados em situações incertas permitindo análise mais fundamentada de riscos e proporcionando a tomada de decisões mais informadas.

A técnica de Monte Carlo será utilizada para o seguinte problema de natureza geométrica: dado um quadrado de lado igual a uma unidade (1) desenha-se dentro dele um quarto de um círculo.

![Imagem Ilustrativa do problema](/assets/image.png)

Ao inserir pontos aletórios dentro do quadrado, alguns pontos podem estar dentro do círculo enquanto outros podem estar fora. A proporção de pontos dentro do círculo em relação ao total é igual a razão entre a área do círculo e a área do quadrado. 

Em termos matemáticos:

$$
\text{ Área do círculo (contida no quadrado) } = \frac{πr^{2}}{4}, r = 1
$$
$$
\text{ Área do quadrado } = 1^2 = 1
$$
$$
\frac{\text{ Área do círculo }}{\text{ Área do quadrado }} = \frac{π}{4}
$$

Logo, se **P** é a probabilidade de um ponto cair dentro do círculo então:

$$
P ≈ \frac{π}{4}
$$
$$
π ≈ 4P  
$$
$$
\text{ ou }
$$
$$
 π ≈ 4 \frac{\text{ Nº de pontos dentro do círculo }}{\text{ Total de pontos }}
$$

Portanto, cabe usar a técnica de Monte Carlo para geração de pontos aleatórios com fim de estimar o valor de π. A seguir serão apresentadas as abordagens serial e paralelizada do problema seguidas de análises a respeito da eficiência de cada implementação.

---

## Implementação

In [16]:
import random
import time
from concurrent.futures import ThreadPoolExecutor

def contar_pontos_circulo(num_pontos):
    dentro_circulo = 0
    for _ in range (num_pontos):
        x = random.random()
        y = random.random()
        if x*x + y*y <=1 :
            dentro_circulo += 1
    return dentro_circulo

### Breve explicação: equação do círculo

A condicional presente no método de contagem dos pontos é baseada na seguinte équação de círculo de raio **r** centrado na origem (0,0):
$$
x^2 + y^2 = r^2
$$

Isso significa que qualquer ponto no plano que satisfaz a seguinte inequação está dentro ou sobre o círculo para r = 1:
$$
x^2 + y^2 \le 1
$$


## Versão serial

In [20]:
if __name__ == "__main__":
    total_pontos = 275_000_000

    inicio = time.time()
    dentro_circulo_total = contar_pontos_circulo(total_pontos)
    pi = 4 * dentro_circulo_total / total_pontos
    fim = time.time()

    resultado_serial = round(fim - inicio,4)

    print(f"π ≈ {pi}")
    print(f"Tempo serial: {fim - inicio:.2f} segundos")


π ≈ 3.1416181236363636
Tempo serial: 48.36 segundos


---

## Versão paralela

In [ ]:
if __name__ == "__main__":
    total_pontos = 275_000_000
    n_processos = 4
    pontos_por_processo = total_pontos // n_processos

    inicio = time.time()
    with ThreadPoolExecutor(max_workers=n_processos) as executor:
        resultados = list(executor.map(contar_pontos_circulo, [pontos_por_processo]*n_processos))
    dentro_circulo_total = sum(resultados)

    pi = 4 * dentro_circulo_total / total_pontos
    fim = time.time()

    resultado_paralelo = round(fim - inicio,4)
    
    print(f"π ≈ {pi}")
    print(f"Tempo paralelo: {fim - inicio:.2f} segundos")

π ≈ 3.141537072
Tempo paralelo: 41.09 segundos


In [21]:
diferenca_resultados = resultado_serial - resultado_paralelo
print(f"Diferença em segundos: {diferenca_resultados}")

Diferença em segundos: 7.271000000000001
